In [ ]:
#数据分为训练集和测试集。训练集包含了10,222张图像，测试集包含了10,357张图像。两个数据集中的图像格式都是JPEG；
#这些图像都含有RGB三个通道（彩色），高和宽的大小不一。训练集中dog的类别共有120种；

In [1]:
# 动物图像分类
# 导入用到的包或模块

import collections
import d2lzh as d2l
import math
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import data as gdata, loss as gloss, model_zoo, nn
import os
import shutil
import time
import zipfile

In [2]:
# 数据整理下面的reorg_train_valid函数来从Kaggle比赛的完整原始训练集中切分出验证集;
# 该函数中的参数valid_ratio指验证集中每类狗的样本数与原始训练集中数量最少一类的狗的样本数（66）之比;


def reorg_train_valid(data_dir, train_dir, input_dir, valid_ratio, idx_label):
    # 训练集中数量最少一类的狗的样本数
    min_n_train_per_label = (
        collections.Counter(idx_label.values()).most_common()[:-2:-1][0][1])# 统计所有种类的狗中数量最少是多少
    # 验证集中每类狗的样本数
    n_valid_per_label = math.floor(min_n_train_per_label * valid_ratio) #每类狗所含有的验证集狗的数量， 每类都是相等的
    label_count = {} #声明 类别统计字典
    for train_file in os.listdir(os.path.join(data_dir, train_dir)):# 遍历 训练集中每张图片
        idx = train_file.split('.')[0] # 当前图片的 idx
        label = idx_label[idx] # 当前图片的类别
        d2l.mkdir_if_not_exist([data_dir, input_dir, 'train_valid', label]) # 在当前路径下创建 label 文件夹
        shutil.copy(os.path.join(data_dir, train_dir, train_file),
                    os.path.join(data_dir, input_dir, 'train_valid', label)) # 把当前图片复制到新创建的label 文件夹中
        if label not in label_count or label_count[label] < n_valid_per_label:
            d2l.mkdir_if_not_exist([data_dir, input_dir, 'valid', label])
            shutil.copy(os.path.join(data_dir, train_dir, train_file),
                        os.path.join(data_dir, input_dir, 'valid', label))# 创建验证集文件夹并复制图片直到满足验证集数量要求
            label_count[label] = label_count.get(label, 0) + 1 #当前label 下数量自增 1
        else:
            d2l.mkdir_if_not_exist([data_dir, input_dir, 'train', label]) # 创建训练集label文件夹， 并复制图像
            shutil.copy(os.path.join(data_dir, train_dir, train_file),
                        os.path.join(data_dir, input_dir, 'train', label))

In [3]:
#下面函数用来读取训练数据标签、切分验证集并整理测试集

def reorg_dog_data(data_dir, label_file, train_dir, test_dir, input_dir,
                   valid_ratio):
    # 读取训练数据标签
    with open(os.path.join(data_dir, label_file), 'r') as f:
        # 跳过文件头行（栏名称）
        lines = f.readlines()[1:]
        tokens = [l.rstrip().split(',') for l in lines]
        idx_label = dict(((idx, label) for idx, label in tokens))
    reorg_train_valid(data_dir, train_dir, input_dir, valid_ratio, idx_label)
    # 整理测试集
    d2l.mkdir_if_not_exist([data_dir, input_dir, 'test', 'unknown'])
    for test_file in os.listdir(os.path.join(data_dir, test_dir)):
        shutil.copy(os.path.join(data_dir, test_dir, test_file),
                    os.path.join(data_dir, input_dir, 'test', 'unknown')) #创建 "unknown"文件夹

In [4]:
# 初始化函数参数值， 这里小批量设置为 batch_size = 32 是由于显存限制， 可调整的大一些

label_file, train_dir, test_dir = 'labels.csv', 'train', 'test'
input_dir, batch_size, valid_ratio = 'train_valid_test', 32, 0.1


In [ ]:
# 开始数据预整理，经过整理后，同一类狗的图像将被放在同一个文件夹下，便于稍后读取;

data_dir = "/home/lzj/桌面/kaggle_dog/"
reorg_dog_data(data_dir, label_file, train_dir, test_dir, input_dir,valid_ratio)

In [5]:
# mxnet 中增广操作，对训练集生成更多的相似图像，加载到模型中，以防止过拟合


transform_train = gdata.vision.transforms.Compose([
    # 随机对图像裁剪出面积为原图像面积0.08~1倍、且高和宽之比在3/4~4/3的图像，再放缩为高和
    # 宽均为224像素的新图像
    gdata.vision.transforms.RandomResizedCrop(224, scale=(0.08, 1.0),
                                              ratio=(3.0/4.0, 4.0/3.0)),#随即裁减操作，大小为（0.08～1.0）之间， 长宽比为（0.75～4.0/3.0z之间）,之后在再放缩为高和
                                            # 宽均为224像素的新图像
    gdata.vision.transforms.RandomFlipLeftRight(), # 随即反转 50% 的概率
    # 随机变化亮度、对比度和饱和度
    gdata.vision.transforms.RandomColorJitter(brightness=0.4, contrast=0.4,
                                              saturation=0.4),
    # 随机加噪声
    gdata.vision.transforms.RandomLighting(0.1),
    gdata.vision.transforms.ToTensor(), # 转成张量，适合model 输入
    # 对图像的每个通道做标准化
    gdata.vision.transforms.Normalize([0.485, 0.456, 0.406],
                                      [0.229, 0.224, 0.225])])# 这里可更改

In [6]:
# 对测试集增广操作（不含随即操作）

transform_test = gdata.vision.transforms.Compose([
    gdata.vision.transforms.Resize(256),
    # 将图像中央的高和宽均为224的正方形区域裁剪出来
    gdata.vision.transforms.CenterCrop(224),
    gdata.vision.transforms.ToTensor(),
    gdata.vision.transforms.Normalize([0.485, 0.456, 0.406],
                                      [0.229, 0.224, 0.225])])

In [7]:
# 通过mxnet中 ImageFolderDataset实例来读取整理后的含原始图像文件的数据集

train_ds = gdata.vision.ImageFolderDataset(
    os.path.join(data_dir, input_dir, 'train'), flag=1)#训练
valid_ds = gdata.vision.ImageFolderDataset(
    os.path.join(data_dir, input_dir, 'valid'), flag=1)#验证
train_valid_ds = gdata.vision.ImageFolderDataset(
    os.path.join(data_dir, input_dir, 'train_valid'), flag=1)#训练+验证
test_ds = gdata.vision.ImageFolderDataset(
    os.path.join(data_dir, input_dir, 'test'), flag=1)#测试

In [8]:
# 批量读取

train_iter = gdata.DataLoader(train_ds.transform_first(transform_train),
                              batch_size, shuffle=True, last_batch='keep')#训练集
valid_iter = gdata.DataLoader(valid_ds.transform_first(transform_test),
                              batch_size, shuffle=True, last_batch='keep')#验证集
train_valid_iter = gdata.DataLoader(train_valid_ds.transform_first(
    transform_train), batch_size, shuffle=True, last_batch='keep') #训练+验证
test_iter = gdata.DataLoader(test_ds.transform_first(transform_test),
                             batch_size, shuffle=False, last_batch='keep')#测试集

In [9]:
#模型定义：使用“微调”，用在ImageNet完整数据集上预训练的模型来抽取图像特征，以作为自定义小规模输出网络的输入
#这里以预训练的ResNet-18模型为例
#因此我们直接复用预训练模型在输出层的输入，即抽取的特征
#然后，我们可以将原输出层替换成自定义的可训练的小规模输出网络，如两个串联的全连接层

def get_net(ctx):
    finetune_net = model_zoo.vision.resnet18_v2(pretrained=True)
    # 定义新的输出网络
    finetune_net.output_new = nn.HybridSequential(prefix='') # 特征提取
    finetune_net.output_new.add(nn.Dense(256, activation='relu')) #自定义隐藏曾
    # 120是输出的类别个数
    finetune_net.output_new.add(nn.Dense(120))#自定义输出层
    # 初始化输出网络
    finetune_net.output_new.initialize(init.Xavier(), ctx=ctx)#在ctx指定设备上参数初始化自定义层
    # 把模型参数分配到内存或显存上
    finetune_net.collect_params().reset_ctx(ctx)
    return finetune_net

In [10]:
loss = gloss.SoftmaxCrossEntropyLoss() # 交叉熵损失

#计算损失时，先通过成员变量features来获取预训练模型输出层的输入，即抽取的特征。
#然后，将该特征作为自定义的小规模输出网络的输入，并计算输出。

def evaluate_loss(data_iter, net, ctx):
    l_sum, n = 0.0, 0
    for X, y in data_iter: #小批量读取
        y = y.as_in_context(ctx)
        output_features = net.features(X.as_in_context(ctx)) #提取的特征
        outputs = net.output_new(output_features) #自定义层输出
        l_sum += loss(outputs, y).sum().asscalar() #当前小批量损失统计， 这里asscalar函数是同步函数， 就是的等待函数
        n += y.size
    return l_sum / n #返回平均损失

In [11]:


def train(net, train_iter, valid_iter, num_epochs, lr, wd, ctx, lr_period,
          lr_decay):
    # 只训练自定义的小规模输出网络
    trainer = gluon.Trainer(net.output_new.collect_params(), 'sgd',
                            {'learning_rate': lr, 'momentum': 0.9, 'wd': wd}) # 优化器
    for epoch in range(num_epochs): #对每个迭代周期进行打印
        train_l_sum, n, start = 0.0, 0, time.time()
        if epoch > 0 and epoch % lr_period == 0: #每隔 lr_period 学习率减小为原来的十分之一
            trainer.set_learning_rate(trainer.learning_rate * lr_decay)
        for X, y in train_iter: #小批量读取
            y = y.as_in_context(ctx)
            output_features = net.features(X.as_in_context(ctx)) # 提取的特征
            with autograd.record():
                outputs = net.output_new(output_features) #只训练自定的输出层
                l = loss(outputs, y).sum() # 当前小批量损失统计
            l.backward() #反向传播
            trainer.step(batch_size) #参数更新
            train_l_sum += l.asscalar() # 当前周期，总损失统计
            n += y.size
        time_s = "time %.2f sec" % (time.time() - start)
        if valid_iter is not None:
            valid_loss = evaluate_loss(valid_iter, net, ctx) #验证集损失计算
            epoch_s = ("epoch %d, train loss %f, valid loss %f, "
                       % (epoch + 1, train_l_sum / n, valid_loss))
        else:
            epoch_s = ("epoch %d, train loss %f, "
                       % (epoch + 1, train_l_sum / n))
        print(epoch_s + time_s + ', lr ' + str(trainer.learning_rate))

In [13]:
# 这里应该进行超参数调整， 以获取最佳超参数
ctx, num_epochs, lr, wd = d2l.try_gpu(), 12, 0.01, 1e-4
lr_period, lr_decay, net = 4, 0.1, get_net(ctx)
net.hybridize()
train(net, train_iter, valid_iter, num_epochs, lr, wd, ctx, lr_period,
      lr_decay)
#假设当前设定超参数就是最优秀超参数

epoch 1, train loss 2.349747, valid loss 1.002689, time 121.95 sec, lr 0.01
epoch 2, train loss 1.311925, valid loss 0.906460, time 138.73 sec, lr 0.01
epoch 3, train loss 1.216916, valid loss 0.851950, time 141.23 sec, lr 0.01
epoch 4, train loss 1.143155, valid loss 0.826127, time 142.16 sec, lr 0.01
epoch 5, train loss 0.969017, valid loss 0.665998, time 137.38 sec, lr 0.001
epoch 6, train loss 0.930508, valid loss 0.638018, time 140.08 sec, lr 0.001
epoch 7, train loss 0.907014, valid loss 0.651266, time 145.49 sec, lr 0.001
epoch 8, train loss 0.902038, valid loss 0.643420, time 142.53 sec, lr 0.001
epoch 9, train loss 0.867955, valid loss 0.638874, time 137.38 sec, lr 0.0001
epoch 10, train loss 0.859323, valid loss 0.635924, time 143.47 sec, lr 0.0001
epoch 11, train loss 0.884376, valid loss 0.634084, time 132.50 sec, lr 0.0001
epoch 12, train loss 0.849597, valid loss 0.635630, time 138.18 sec, lr 0.0001


In [ ]:
#事实上，在得到一组满意的模型设计和超参数后，因该使用全部训练数据集（含验证集）重新训练模型，并对测试集分类。

net = get_net(ctx)
net.hybridize() #mxnet 符号编程
train(net, train_valid_iter, None, num_epochs, lr, wd, ctx, lr_period,
      lr_decay) #用整个数据集 train_valid 重新训练模型

# ----------------------------------------------------------------------#
# 生成 提交文件 submission.csv
preds = []
for data, label in test_iter:
    output_features = net.features(data.as_in_context(ctx)) # test 数据提取的特征
    output = nd.softmax(net.output_new(output_features)) # softamx 运算
    preds.extend(output.asnumpy()) # 对每张测试集里的图像最预测并将结果转成 numpy 格式（应为预测是ndarray格式）
ids = sorted(os.listdir(os.path.join(data_dir, input_dir, 'test/unknown'))) 
with open('submission.csv', 'w') as f:
    f.write('id,' + ','.join(train_valid_ds.synsets) + '\n')
    for i, output in zip(ids, preds):
        f.write(i.split('.')[0] + ',' + ','.join(
            [str(num) for num in output]) + '\n')